In [1]:
import os.path
has_train_data = os.path.isfile('train.p') 
if not has_train_data:
    from zipfile import ZipFile
    import urllib.request
    from tempfile import mktemp

    zipurl = 'https://d17h27t6h515a5.cloudfront.net/topher/2016/November/581faac4_traffic-signs-data/traffic-signs-data.zip'
    from urllib.request import urlopen
    from zipfile import ZipFile
    zipresp = urlopen(zipurl)
    tempzip = open("/tmp/tempfile.zip", "wb")
    tempzip.write(zipresp.read())
    tempzip.close()
    zf = ZipFile("/tmp/tempfile.zip")
    zf.extractall()
    zf.close()

In [2]:
import pickle
train_data = 'train.p'
test_data = 'test.p'
with open(train_data, 'rb') as f:
    pickle_data = pickle.load(f)
    train_features = pickle_data['features']
    train_labels = pickle_data['labels']
    del pickle_data  # Free up memory
with open(test_data, 'rb') as f:
    pickle_data = pickle.load(f)
    test_features = pickle_data['features']
    test_labels = pickle_data['labels']
    del pickle_data  # Free up memory

In [3]:
import cv2
import numpy as np
gray_train_features = [cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in  train_features]
gray_test_features = [cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in  test_features]


In [4]:
def normalize_greyscale(image, a=0.1, b=0.9,grey_min=0,grey_max=255):
    return a + ( ( (image - grey_min)*(b - a) )/( grey_max - grey_min ) )
train_features = [normalize_greyscale(image) for image in gray_train_features]
test_features = [normalize_greyscale(image) for image in gray_test_features]

In [5]:
from sklearn.utils import shuffle
train_features, train_labels = shuffle(train_features,train_labels, random_state=0)

In [6]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
encoder.fit(train_labels)
train_labels = encoder.transform(train_labels)
test_labels = encoder.transform(test_labels)

In [7]:
# Change to float32, so it can be multiplied against the features in TensorFlow, which are float32
train_labels = train_labels.astype(np.float32)
test_labels = test_labels.astype(np.float32)

In [8]:
import tensorflow as tf

# Parameters
learning_rate = 0.01
training_epochs = 100
batch_size = 100
display_step = 5


n_input = 1024
n_classes = 43


n_hidden_layer = 512



weights = {
    'hidden_layer': tf.Variable(tf.random_normal([n_input, n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_hidden_layer, n_classes]))
}
biases = {
    'hidden_layer': tf.Variable(tf.random_normal([n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# tf Graph input
x = tf.placeholder("float", [None, 32, 32])
y = tf.placeholder("float", [None, n_classes])

x_flat = tf.reshape(x, [-1, n_input])


# Hidden layer with RELU activation
layer_1 = tf.add(tf.matmul(x_flat, weights['hidden_layer']), biases['hidden_layer'])

relu_layer = tf.nn.relu(layer_1)
# Output layer with linear activation
logits = tf.matmul(relu_layer, weights['out']) + biases['out']

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)


In [9]:
# Initializing the variables
init = tf.initialize_all_variables()


# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(train_features)/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_start = i*batch_size
            batch_features = train_features[batch_start:batch_start + batch_size]
            batch_labels = train_labels[batch_start:batch_start + batch_size]
            #sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_features, y: batch_labels})
            # Compute average loss
            avg_cost += c / total_batch
            
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print ("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    print("Accuracy on testing data:", accuracy.eval({x: test_features, y: test_labels}))
    print("Accuracy on training data:", accuracy.eval({x: train_features, y: train_labels}))

Epoch: 0005 cost= 13.373128435
Epoch: 0010 cost= 7.121759761
Epoch: 0015 cost= 4.879999573
Epoch: 0020 cost= 3.705289567
Epoch: 0025 cost= 2.976878666
Epoch: 0030 cost= 2.475927327
Epoch: 0035 cost= 2.112638167
Epoch: 0040 cost= 1.839081452
Epoch: 0045 cost= 1.618227084
Epoch: 0050 cost= 1.441208685
Epoch: 0055 cost= 1.298981960
Epoch: 0060 cost= 1.174353900
Epoch: 0065 cost= 1.071224629
Epoch: 0070 cost= 0.981259308
Epoch: 0075 cost= 0.903629955
Epoch: 0080 cost= 0.834333138
Epoch: 0085 cost= 0.774971789
Epoch: 0090 cost= 0.720965229
Epoch: 0095 cost= 0.673354762
Epoch: 0100 cost= 0.630937833
Optimization Finished!
Accuracy on testing data: 0.729295
Accuracy on training data: 0.902369
